### Import Modules

In [1]:
import numpy as np
import pandas as pd

In [2]:
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [3]:
import requests
from bs4 import BeautifulSoup

### Download the Data

In [25]:
url = 'http://ncov.mohw.go.kr/bdBoardList_Real.do?brdId=1&brdGubun=13&ncvContSeq=&contSeq=&board_id=&gubun='
html_text = requests.get(url).text

soup = BeautifulSoup(html_text, 'html.parser')
tables = soup.select('#content > div > div.data_table.midd.mgt24 > table')

table = tables[0]
table_html = str(table)
table_df_list = pd.read_html(table_html)
table_df = table_df_list[0]
table_df.drop([0, 18], inplace = True)
table_df.columns = ['시도명', '합계', '해외유입', '국내발생', '확진환자', '격리중', '격리헤제', '사망자', '발생률 (*)']
table_df

,시도명,합계,해외유입,국내발생,확진환자,격리중,격리헤제,사망자,발생률 (*)
1,서울,26,1,25,1735,166,1556,13,17.83
2,부산,2,0,2,193,30,160,3,5.66
3,대구,1,1,0,6946,6,6749,191,285.08
4,인천,0,0,0,389,12,374,3,13.16
5,광주,2,1,1,218,29,187,2,14.97
6,대전,0,0,0,167,1,164,2,11.33
7,울산,1,0,1,61,4,56,1,5.32
8,세종,0,0,0,50,0,50,0,14.61
9,경기,17,1,16,1681,186,1463,32,12.69
10,강원,0,0,0,77,4,70,3,5.00


In [28]:
from pandas import DataFrame as df

city_name = df(data = {'시도명':['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북', '충남', '전북',\
                              '전남', '경북', '경남', '제주'], '이름':['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', \
                                                            '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도', \
                                                            '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도']})

final_df = pd.merge(table_df, city_name)
final_df

,시도명,합계,해외유입,국내발생,확진환자,격리중,격리헤제,사망자,발생률 (*),이름
0,서울,26,1,25,1735,166,1556,13,17.83,서울특별시
1,부산,2,0,2,193,30,160,3,5.66,부산광역시
2,대구,1,1,0,6946,6,6749,191,285.08,대구광역시
3,인천,0,0,0,389,12,374,3,13.16,인천광역시
4,광주,2,1,1,218,29,187,2,14.97,광주광역시
5,대전,0,0,0,167,1,164,2,11.33,대전광역시
6,울산,1,0,1,61,4,56,1,5.32,울산광역시
7,세종,0,0,0,50,0,50,0,14.61,세종특별자치시
8,경기,17,1,16,1681,186,1463,32,12.69,경기도
9,강원,0,0,0,77,4,70,3,5.00,강원도


### Create a Map

### - Cumulative status

In [40]:
center = [35.95, 128.25]
map1 = folium.Map(location = center, zoom_start = 7)
geo_data = 'https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2013/json/skorea_provinces_geo_simple.json'

map1.choropleth(
    geo_data = geo_data,
    data = final_df,
    columns = ['이름', '확진환자'],
    key_on = 'feature.properties.name',
    fill_color = 'YlOrRd',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name = 'Confirmed Patient'
)

map1

### - Daily status

In [41]:
map2 = folium.Map(location = center, zoom_start = 7)

map2.choropleth(
    geo_data = geo_data,
    data = final_df,
    columns = ['이름', '합계'],
    key_on = 'feature.properties.name',
    fill_color = 'YlOrRd',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name = 'Confirmed Patient'
)

map2